# Neural Network Part2

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from datetime import date
date.today()

datetime.date(2017, 3, 22)

In [3]:
author = "kyubyong. https://github.com/Kyubyong/tensorflow-exercises"

In [4]:
tf.__version__

'1.0.0'

In [5]:
np.__version__

'1.12.0'

# Normalization

Q1. Apply `l2_normalize` to `x`.

In [20]:
_x = np.arange(1, 11)
epsilon = 1e-12
x = tf.convert_to_tensor(_x, tf.float32)

output = tf.nn.l2_normalize(x, dim=0, epsilon=epsilon)
with tf.Session() as sess:
    _output = sess.run(output)

    assert np.allclose(_output, _x / np.sqrt(np.maximum(np.sum(_x**2), epsilon)))
print(_output)

[ 0.05096472  0.10192944  0.15289417  0.20385888  0.2548236   0.30578834
  0.35675305  0.40771776  0.45868248  0.50964719]


Q2. Calculate the mean and variance of `x` based on the sufficient statistics.

In [40]:
_x = np.arange(1, 11)
x = tf.convert_to_tensor(_x, tf.float32)

counts_, sum_, sum_of_squares_, _ = tf.nn.sufficient_statistics(x, [0])
mean, variance = tf.nn.normalize_moments(counts_, sum_, sum_of_squares_, shift=None)
with tf.Session() as sess:
    _mean, _variance = sess.run([mean, variance])
print(_mean, _variance)


5.5 8.25


Q3. Calculate the mean and variance of `x`.

In [39]:
tf.reset_default_graph()
_x = np.arange(1, 11)
x = tf.convert_to_tensor(_x, tf.float32)

output = tf.nn.moments(x, [0])
with tf.Session() as sess:
    _mean, _variance = sess.run(output)
print(_mean, _variance)

5.5 8.25


Q4. Calculate the mean and variance of `x` using `unique_x` and `counts`.

In [63]:
tf.reset_default_graph()
x = tf.constant([1, 1, 2, 2, 2, 3], tf.float32)

# From `x`
mean, variance = tf.nn.moments(x, [0])
with tf.Session() as sess:
    print(sess.run([mean, variance]))

# From unique elements and their counts
unique_x, _, counts = tf.unique_with_counts(x)
mean, variance = tf.nn.weighted_moments(unique_x, [0], counts)
with tf.Session() as sess:
    print(sess.run([mean, variance]))


[1.8333334, 0.47222227]
[1.8333334, 0.47222221]


Q5. The code below is to implement the mnist classification task. Complete it by adding batch normalization.

In [16]:
# Load data 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)

# build graph
class Graph:
    def __init__(self, is_training=False):
        # Inputs and labels
        self.x = tf.placeholder(tf.float32, shape=[None, 784])
        self.y = tf.placeholder(tf.int32, shape=[None])

        # Layer 1
        w1 = tf.get_variable("w1", shape=[784, 100], initializer=tf.truncated_normal_initializer())
        output1 = tf.matmul(self.x, w1)
        output1 = tf.contrib.layers.batch_norm(output1, center=True, scale=True, is_training=is_training, 
                                           updates_collections=None, activation_fn=tf.nn.relu)

        #Layer 2
        w2 = tf.get_variable("w2", shape=[100, 10], initializer=tf.truncated_normal_initializer())
        logits = tf.matmul(output1, w2)
        preds = tf.to_int32(tf.arg_max(logits, dimension=1))

        # training
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.y, logits=logits)
        self.train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
        self.acc = tf.reduce_mean(tf.to_float(tf.equal(self.y, preds)))


# Training
tf.reset_default_graph()
g = Graph(is_training=True)
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    saver = tf.train.Saver()
    for i in range(1, 10000+1):
        batch = mnist.train.next_batch(60)
        sess.run(g.train_op, {g.x: batch[0], g.y: batch[1]})
        # Evaluation
        if i % 100 == 0:
            print("training steps=", i, "Acc. =", sess.run(g.acc, {g.x: mnist.test.images, g.y: mnist.test.labels}))
    save_path = saver.save(sess, './my-model')
        
# Inference
tf.reset_default_graph()
g2 = Graph(is_training=False)
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, save_path)
    hits = 0
    for i in range(100):
        hits += sess.run(g2.acc, {g2.x: [mnist.test.images[i]], g2.y: [mnist.test.labels[i]]})
    print(hits)





Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
training steps= 100 Acc. = 0.7739
training steps= 200 Acc. = 0.8215
training steps= 300 Acc. = 0.8529
training steps= 400 Acc. = 0.8649
training steps= 500 Acc. = 0.879
training steps= 600 Acc. = 0.8804
training steps= 700 Acc. = 0.8874
training steps= 800 Acc. = 0.8981
training steps= 900 Acc. = 0.8952
training steps= 1000 Acc. = 0.8998
training steps= 1100 Acc. = 0.9033
training steps= 1200 Acc. = 0.9087
training steps= 1300 Acc. = 0.9101
training steps= 1400 Acc. = 0.9084
training steps= 1500 Acc. = 0.9161
training steps= 1600 Acc. = 0.9165
training steps= 1700 Acc. = 0.9168
training steps= 1800 Acc. = 0.9222
training steps= 1900 Acc. = 0.9212
training steps= 2000 Acc. = 0.9248
training steps= 2100 Acc. = 0.9272
training steps= 2200 Acc. = 0.9258
training steps= 2300 Acc. = 0.9274
training st

# Losses

Q06. Compute half the L2 norm of `x` without the sqrt.


In [89]:
tf.reset_default_graph()
x = tf.constant([1, 1, 2, 2, 2, 3], tf.float32)

output = tf.nn.l2_loss(x)
with tf.Session() as sess:
    print(sess.run(output))
    print(sess.run(tf.reduce_sum(x**2)/2))

11.5
11.5


# Classification

Q7. Compute softmax cross entropy between logits and labels. Note that the rank of them is not the same.


In [108]:
tf.reset_default_graph()
logits = tf.random_normal(shape=[2, 5, 10])
labels = tf.convert_to_tensor(np.random.randint(0, 10, size=[2, 5]), tf.int32)
output = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print(sess.run(output))

[[ 3.20745516  2.13898396  3.07336521  2.74353743  3.27086067]
 [ 3.30273581  2.07290959  3.74236178  2.16736674  2.34212399]]


Q8. Compute softmax cross entropy between logits and labels.

In [110]:
logits = tf.random_normal(shape=[2, 5, 10])
labels = tf.convert_to_tensor(np.random.randint(0, 10, size=[2, 5]), tf.int32)
labels = tf.one_hot(labels, depth=10)

output = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print(sess.run(output))

[[ 4.10439205  3.70805836  3.14382291  2.34843588  3.06786823]
 [ 2.46797109  1.83231926  1.75755191  4.17633867  4.40706158]]


# Embeddings

Q9. Map tensor `x` to the embedding.

In [113]:
tf.reset_default_graph()
x = tf.constant([0, 2, 1, 3, 4], tf.int32)
embedding = tf.constant([0, 0.1, 0.2, 0.3, 0.4], tf.float32)
output = tf.nn.embedding_lookup(embedding, x)
with tf.Session() as sess:
    print(sess.run(output))

[ 0.          0.2         0.1         0.30000001  0.40000001]
